In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('..')

In [10]:
import os
from bs4 import BeautifulSoup
from src.utils.data import create_dataset, split_metadata
from src.utils.data import xml_to_dataframe
from tensorflow import keras

dataset_path = '../data/processed/dataset'
annotations_path = os.path.join(dataset_path, 'annotations.xml')
images_path = os.path.join(dataset_path, 'images')

with open(annotations_path, 'rb') as f:
    annotations = BeautifulSoup(f.read(), 'xml')

df = xml_to_dataframe(annotations)
df['file_path'] = images_path + '/' + df['file_name']


# Split the data to train and test
train_images, valid_images, test_images = split_metadata(df, train_frac=0.7, valid_frac=0.15)

# Create datasets
ds_train = create_dataset(train_images)
ds_valid = create_dataset(valid_images)
ds_test = create_dataset(test_images)

# Create model
backbone = keras.applications.Xception(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=(256, 256, 3),
    include_top=False)

backbone.trainable = False
inputs = keras.Input(shape=(256, 256, 3))

# We make sure that the base_model is running in inference mode here,
# by passing `training=False`. This is important for fine-tuning, as you will
# learn in a few paragraphs.
x = backbone(inputs, training=False)

# Convert features of shape `base_model.output_shape[1:]` to vectors
x = keras.layers.GlobalAveragePooling2D()(x)

# A Dense classifier with a single unit (binary classification)
x = keras.layers.Dense(512, activation='relu')(x)
outputs = keras.layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

In [11]:
model.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_10 (InputLayer)       [(None, 256, 256, 3)]     0         
                                                                 
 xception (Functional)       (None, 8, 8, 2048)        20861480  
                                                                 
 global_average_pooling2d_4   (None, 2048)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_6 (Dense)             (None, 512)               1049088   
                                                                 
 dense_7 (Dense)             (None, 1)                 513       
                                                                 
Total params: 21,911,081
Trainable params: 1,049,601
Non-trainable params: 20,861,480
_______________________________________